In [1]:
import torch
import numpy as np 
import os 
import diffusers
import transformers
import shutil
from IPython.display import display

In [ ]:
!git clone https://github.com/kohya-ss/sd-scripts.git

In [ ]:
image_dir = "image_data_dir"
photo_caption = "a photo of owhn man"

In [ ]:
input_image_dir = "image_caption_dir"
os.makedirs(input_image_dir, exist_ok = True)
for filename in os.listdir(image_dir):
    source_image_path = os.path.join(image_dir, filename)
    base_name, extension = os.path.splitext(filename)
    caption_file = base_name + ".txt"
    dest_txt_path = os.path.join(input_image_dir, caption_file)
    dest_image_path = os.path.join(input_image_dir, filename)

    shutil.copy2(source_image_path, dest_image_path)
    with open(dest_txt_path, 'w', encoding = 'utf-8') as f:
        f.write(photo_caption)

In [ ]:
model_dir = "path_to_save_model"
os.makedirs(model_dir, exist_ok = True)

In [ ]:
log_dir = "./log"
sample_prompt = "(wide shot) of analog style modelshoot style 8k close up linkedin profile picture of owhn man, professional jack suite, professional headshots, photo-realistic, 4k, high-resolution image, upper body, modern outfit, business, blurred background, glass building background, crisp details, neutral expression, high-definition, sharp focus"

sample_prompt_dir = "sample_prompt_path"
with open(sample_prompt_dir, 'w', encoding = 'utf-8') as f:
    f.write(sample_prompt)

In [ ]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="stabilityai/stable-diffusion-xl-base-1.0",
    filename="sd_xl_base_1.0.safetensors",  # hoặc file bạn muốn
    local_dir="./sdxl_model"
)

print("Downloaded to:", model_path)

In [ ]:
!pip install voluptuous bitsandbytes

In [9]:
os.makedirs(log_dir, exist_ok = True)

In [ ]:
file_path = 'file_path'

if os.path.exists(file_path):
    os.remove(file_path)
    print("File deleted successfully.")
else:
    print("File does not exist.")

In [11]:
toml_text = '''
[[datasets]]
resolution = 1024
batch_size = 1
keep_tokens = 2

    [[datasets.subsets]]
    image_dir = "/kaggle/working/itay_with_caption"
    class_tokens = 'owhn man'
    caption_extension = '.txt'
    num_repeats = 5

'''

with open("dataset_config.toml", "w") as f:
    f.write(toml_text.strip())

In [ ]:
!accelerate launch --num_cpu_threads_per_process 1 /kaggle/working/sd-scripts/sdxl_train_network.py \
    --pretrained_model_name_or_path="/kaggle/working/sdxl_model/sd_xl_base_1.0.safetensors" \
    --dataset_config="image_caption_dir" \
    --output_dir="path_to_save_model" \
    --output_name="owhn man" \
    --logging_dir="log_dir" \
    --max_token_length=225 \
    --max_train_epochs=20 \
    --save_every_n_epochs=4 \
    --learning_rate=1e-4 \
    --mixed_precision="bf16" \
    --cache_latents \
    --gradient_checkpointing \
    --optimizer_type="AdamW8bit" \
    --lr_scheduler="constant" \
    --sample_every_n_epochs=4 \
    --sample_prompt="sample_prompt_path" \
    --save_model_as=safetensors \
    --sample_sampler="euler_a" \
    --seed=0 \
    --log_with="wandb" \
    --wandb_api_key="wandb_api_key" \
    --network_dim=64 \
    --unet_lr=0.0001 \
    --text_encoder_lr=5e-6 \
    --network_module="networks.lora" \
    --network_alpha=64